In [2]:
import numpy as np
import sys

In [3]:
a = 10
b = np.binary_repr(a)
b = list(str(b))
b[2] = "0"
print(b)
b = "".join(b)
print(int(b,2))
for state in b:
    print(state=="1")

['1', '0', '0', '0']
8
True
False
False
False


In [4]:
def create_fermionic_operator(i, j, N):
    if i > j:
        raise ValueError("i must be smaller or equal to j")
    else:
        L = 2**N
        H = np.zeros((L,L))
        if i!=j:
            for k in range(L):
                state = list(np.binary_repr(k, N))
                if state[i] == "0" and state[j] == "1":
                    counter = 1
                    for l in range(i, j):
                        cur_str = state[l]
                        if cur_str == "1":
                            counter *= -1
                    new_state = state.copy()
                    new_state[i] = "1"
                    new_state[j] = "0"
                    new_state = "".join(new_state)
                    new_state = int(new_state, 2)
                    H[new_state, k] = counter
        else:
            for k in range(L):
                state = list(np.binary_repr(k, N))
                if state[i] == "1":
                    H[k, k] = 1
    H = H - H.transpose()
    return H

def create_fermionic_operator_test(i, j, N):
    if i > j:
        raise ValueError("i must be smaller or equal to j")
    else:
        L = 2**N
        H = np.zeros((L,L))
        if i!=j:
            for k in range(L):
                state = list(np.binary_repr(k, N))
                "(a^+_i a_j)"
                if state[i] == "0" and state[j] == "1":
                    counter = 1
                    for l in range(i, j):
                        cur_str = state[l]
                        if cur_str == "1":
                            counter *= -1
                    new_state = state.copy()
                    new_state[i] = "1"
                    new_state[j] = "0"
                    new_state = "".join(new_state)
                    new_state = int(new_state, 2)
                    H[new_state, k] = counter
                "-(a^+_j a_i)"
                if state[j] == "0" and state[i] == "1":
                    counter = -1
                    for l in range(i+1, j):
                        cur_str = state[l]
                        if cur_str == "1":
                            counter *= -1
                    
                    new_state = state.copy()
                    new_state[i] = "0"
                    new_state[j] = "1"
                    new_state = "".join(new_state)
                    # print(state, "old")
                    # print(new_state, "new")
                    # print(counter)
                    # print("=====")
                    new_state = int(new_state, 2)
                    H[new_state, k] = counter
        else:
            for k in range(L):
                state = list(np.binary_repr(k, N))
                if state[i] == "1":
                    H[k, k] = 1
    return H

In [5]:
i = 0
j = 9
N = 10
H = create_fermionic_operator(i,j,N)
H1 = create_fermionic_operator_test(i,j,N)
print(f"The size of system is {N}, i={i}, j={j}")
print(f"The difference between H and H1 is {np.sum(abs(H-H1))}")
# print(H)
# print(H1)


The size of system is 10, i=0, j=9
The difference between H and H1 is 0.0


In [6]:
def create_qubit_operator(i,j,N):
    if i>j:
        raise ValueError("i must be smaller or equal to j")
    L = 2**N
    H = np.zeros((L,L))
    for k in range(L):
        state = list(np.binary_repr(k, N))
        if state[i] == "0" and state[j] == "1":
            counter = 1
            for l in range(i, j):
                cur_str = state[l]
                if cur_str == "1":
                    counter *= -1
            new_state = state.copy()
            new_state[i] = "1"
            new_state[j] = "0"
            new_state = "".join(new_state)
            new_state = int(new_state, 2)
            H[new_state, k] = counter
    H = H - H.transpose()
    return H

In [22]:
from scipy.linalg import expm
i = 0
j = 1
N = 2
t = 1
H = create_qubit_operator(i,j,N)
print(expm(H))
y = np.array([[0, -1j], [1j, 0]])
x = np.array([[0, 1], [1, 0]])
z = np.array([[1, 0], [0, -1]])
A = np.kron(-1j*x/2,z)
print(A)


[[ 1.          0.          0.          0.        ]
 [ 0.          0.54030231 -0.84147098  0.        ]
 [ 0.          0.84147098  0.54030231  0.        ]
 [ 0.          0.          0.          1.        ]]
[[0.+0.j  0.+0.j  0.-0.5j 0.+0.j ]
 [0.+0.j  0.+0.j  0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j  0.+0.j  0.+0.j ]
 [0.+0.j  0.+0.5j 0.+0.j  0.+0.j ]]


In [24]:
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
def construct_circuit(i,j,N,t):
    # Construc before hand gates
    qc = QuantumCircuit(N)
    qc.h(i)

    for k in range(i,j):
        qc.cx(k, k+1)
    qc.rz(t, j)
    for k in range(j,i,-1):
        qc.cx(k-1, k)
        print(k)

    qc.h(i)    
    return qc

i = 0
j = 1
N = 2
t = 1
qc = construct_circuit(i,j,N,t)
print(qc)
M = qi.Operator(qc).to_matrix()
M = np.array(M)
print(M)
print("///")
H = expm(A)
print(abs(H-M))


1
     ┌───┐                   ┌───┐
q_0: ┤ H ├──■─────────────■──┤ H ├
     └───┘┌─┴─┐┌───────┐┌─┴─┐└───┘
q_1: ─────┤ X ├┤ Rz(1) ├┤ X ├─────
          └───┘└───────┘└───┘     
[[0.87758256+0.j         0.        -0.47942554j 0.        +0.j
  0.        +0.j        ]
 [0.        -0.47942554j 0.87758256+0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.87758256+0.j
  0.        +0.47942554j]
 [0.        +0.j         0.        +0.j         0.        +0.47942554j
  0.87758256+0.j        ]]
///
[[2.22044605e-16 4.79425539e-01 4.79425539e-01 0.00000000e+00]
 [4.79425539e-01 2.22044605e-16 0.00000000e+00 4.79425539e-01]
 [4.79425539e-01 0.00000000e+00 2.22044605e-16 4.79425539e-01]
 [0.00000000e+00 4.79425539e-01 4.79425539e-01 2.22044605e-16]]


In [28]:
test = QuantumCircuit(2, name="hi")
# test.sdg(0)
# test.h(0)
# test.z(0)
# test.h(0)
# test.s(0)
test.x(0)
test.z(1)
m = qi.Operator(test)
print(m)
print(test)
print(np.kron(z,x))

Operator([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
          [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
          [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ Z ├
     └───┘
[[ 0  1  0  0]
 [ 1  0  0  0]
 [ 0  0  0 -1]
 [ 0  0 -1  0]]


In [10]:
N = 2
qc = QuantumCircuit(N)
i = 0
j = 1
for k in range(i,j):
    qc.cx(k, k+1)
qc.rz(-1, j)
for k in range(j,i,-1):
    qc.cx(k-1, k)
print(qc)
m = qi.Operator(qc)
print(m)


                         
q_0: ──■──────────────■──
     ┌─┴─┐┌────────┐┌─┴─┐
q_1: ┤ X ├┤ Rz(-1) ├┤ X ├
     └───┘└────────┘└───┘
Operator([[0.87758256+0.47942554j, 0.        +0.j        ,
           0.        +0.j        , 0.        +0.j        ],
          [0.        +0.j        , 0.87758256-0.47942554j,
           0.        +0.j        , 0.        +0.j        ],
          [0.        +0.j        , 0.        +0.j        ,
           0.87758256-0.47942554j, 0.        +0.j        ],
          [0.        +0.j        , 0.        +0.j        ,
           0.        +0.j        , 0.87758256+0.47942554j]],
         input_dims=(2, 2), output_dims=(2, 2))


In [11]:
Z = np.array([[1,0], [0,-1]])
Z2 = np.kron(Z,Z)
print(expm(1j/2*Z2))

[[0.87758256+0.47942554j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.87758256-0.47942554j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.87758256-0.47942554j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.87758256+0.47942554j]]


In [1]:
import sympy as sp

# Define symbols
a1, a2, b1, b2, c1, c2, d1, d2 = sp.symbols('a1 a2 b1 b2 c1 c2 d1 d2')
i = sp.I  # imaginary unit

# Define the expression
expr = -((a1 - i*a2)*(b1 - i*b2)*(c1 + i*c2)*(d1 + i*d2)) + \
       ((a1 + i*a2)*(b1 + i*b2)*(c1 - i*c2)*(d1 - i*d2))

# Factor the expression
factored_expr = sp.factor(expr)

print(factored_expr)

-2*I*(a1*b1*c1*d2 + a1*b1*c2*d1 - a1*b2*c1*d1 + a1*b2*c2*d2 - a2*b1*c1*d1 + a2*b1*c2*d2 - a2*b2*c1*d2 - a2*b2*c2*d1)
